# Editing CreateLib

Now that you have your robot moving - it might be nice to change things.  The CreateLib is a library that produces a bunch of blocking commands (that is you do not move to the next line until the previous line is finished executing).  It would be nice, for instance, to dictate the speed of rotation as well as the angle. Lets start by looking at the start of the file:
```python
import rclpy
from ROS2Lib import Drive, Rotate, Lights, Audio
from TCPLib import TCPServer
import time
```

You can see we *import rclpy* - that is the library from the ROS folks - that talks ROS.  Then we import a bunch of functions from ROS2Lib - you will be editing this library later in the year.  The TCP library is how you will talk with stuff we physically stick on top of the Create.  We will now define a class called Create.  What is cool about classes is that you can call the same code many different times to control different objects.  For instance, if you have 3 creates, you can have a code that looks like this - each instance (fred, sara, ethel) keeps track of their own internal variables (that is what *self* is all about).
```python
fred = Create('/rogers')
sara = Create('/dory')
ethel = Create('/woody')

fred.forward(1)
ethel.turn(90)
sara.beep()
```

So lets we define the class Create - when called, it will run *init* first -and save a bunch of info that will belong to the instance of the class  (fred, sara, or ethel)
```python
class Create():
    def __init__ (self, namespace = ''):
        rclpy.init(args = None)
        self.namespace = namespace
        self.drive_client = Drive(namespace)
        self.rotate_client = Rotate(namespace)
        self.led_publisher = Lights(namespace)
        self.audio_publisher = Audio(namespace)
        self.serial = None
        time.sleep(1)
```

next lets define a method to change the LED colors.  There are 12 colors predetermined (there could be a lot more if you want to edit ROSLib).  So we read in the color parameter (number from 0-11) and publish (a ROS [publisher](104-ROS.ipynb)) it out to the Create.  We print some info to help debug for the console and then we wait for a second for it all to happen.  Note that comments can be in triple quotes or with a hashtag.
```python
    def LED(self,color):
        '''
        changes the color of the LED
        '''
        led_colors = color
        print('publish LED ', end = ''        )
        self.led_publisher.set_color(led_colors)
        time.sleep(1)
        print('done')
```

Similarly, we can publish a beep with a given frequency
```python
    def beep(self, frequency = 440):
        '''
        Beeps
        '''
        print('publish beep ', end = ''        )
        self.audio_publisher.beep(frequency)
        time.sleep(1)
        print('done')
```

Getting a little more complicated, we can define a turn
```python
    def turn(self,angle = 90):
        '''
        rotates a given angle
        '''
        speed = 0.5   
        angle = angle/180*3.1415
        print('turn %0.2f: goal' % angle, end = '')
        self.rotate_client.set_goal(float(angle), speed)
        print(' set ', end = '')
        self.wait(self.rotate_client)
        print('done')
```

Notice that we first define a turning speed, convert the angle to radians, and print out that we are setting a goal.  This uses a ROS [action](204-ROS.ipynb) - so we tell the action the goal, and then we wait for the goal to complete (and print done).  Try editing Create so that you can pass in both the angle and the speed - and pass that on into the goal..  Make sure to edit and save the CreateLib.py and then you can test it with your original code in the first notebook.  We will talk about the serial calls in a later notebook.

*hint: you can give parameters default values in python - note that the default value for the angle is 90 - this means if they do not enter a number when calling turn (fred.turn()), it will turn 90 degrees - do the same for the speed*

**Editing CreateLib.py** 
<br>You can edit the CreateLib.py either directly (open the python file and remember to hit save before restarting the kernel and running it) or we can leverage the Jupyter command of *%%writefile* and run the cell below - then restart the kernel and run a test.<br><br>
Try adding speed control to the forward command:
- expand the Create class (little grey arrow to the side)
- scroll down to def forward(self, dist = 0.5):
- change it to def forward(self, dist = 0.5, speed = 0.25):
- delete the next line where speed is defined: speed = 0.25
- save
- try the code two cells below - does it pass in it move faster?  Note that you could also still have the line be forward(0.5) and it would use a speed of 0.25 (default).

In [2]:
%%writefile CreateLib.py
'''
This library talks to the ROS library, setting up some key behaviors
'''

import rclpy
from ROS2Lib import Drive, Rotate, Lights, Audio
from TCPLib import TCPServer
import time

class Create():
    def __init__ (self, namespace = ''):
        rclpy.init(args = None)
        self.namespace = namespace
        self.drive_client = Drive(namespace)
        self.rotate_client = Rotate(namespace)
        self.led_publisher = Lights(namespace)
        self.audio_publisher = Audio(namespace)
        self.serial = None
        time.sleep(1)

    def LED(self,color):
        '''
        changes the color of the LED
        '''
        led_colors = color
        print('publish LED ', end = ''        )
        self.led_publisher.set_color(led_colors)
        time.sleep(1)
        print('done')

    def beep(self, frequency = 440):
        '''
        Beeps
        '''
        print('publish beep ', end = ''        )
        self.audio_publisher.beep(frequency)
        time.sleep(1)
        print('done')
            
    def turn(self,angle = 90):
        '''
        rotates a given angle
        '''
        speed = 0.5   
        angle = angle/180*3.1415
        print('turn %0.2f: goal' % angle, end = '')
        self.rotate_client.set_goal(float(angle), speed)
        print(' set ', end = '')
        self.wait(self.rotate_client)
        print('done')

    def forward(self,dist = 0.5):
        '''
        goes the distance and then stops the ROS2 connection
        '''
        speed = 0.25
        print('forward %0.2f: goal' % dist, end = '')
        self.drive_client.set_goal(float(dist),speed)
        print(' set ', end = '')
        self.wait(self.drive_client)
        print('done')
        
    def wait(self, client):
        rclpy.spin_once(client)
        while not client.done:
            #time.sleep(0.1)
            print('...', end = '')
            rclpy.spin_once(client)

# ----------------------------------------serial calls using serial over TCP------------------------- 

    def serial_init(self, IP, PORT, timeout = 0):
        self.serial = TCPServer (IP, PORT, timeout)
        
    def serial_write(self, string):
        if self.serial:
            self.serial.write(string)
        else:
            print('serial not initialized')
            
    def serial_write_binary(self, string):
        if self.serial:
            self.serial.write_binary(string)
        else:
            print('serial not initialized')
            
    def serial_abort(self):
        self.serial.write_binary(b'\x03')
            
    def serial_run(self, code):
        code = code.replace('\n','\r\n')
        code = code.replace('\t','    ')
        if self.serial:
            self.serial.write_binary(b'\x05') # Ctrl E
            self.serial.write(code)
            self.serial.write_binary(b'\x04')  #Ctrl D
        else:
            print('serial not initialized')
            
    def serial_read(self):
        if self.serial:
            return self.serial.read()
        else:
            print('serial not initialized')
        return None
        
    def serial_close(self):
        if self.serial:
            return self.serial.close()
        else:
            print('serial not initialized')

    def close(self):
        self.drive_client.destroy_node()
        self.rotate_client.destroy_node()
        self.led_publisher.destroy_node()
        self.audio_publisher.destroy_node()
        rclpy.shutdown()



Overwriting CreateLib.py


In [ ]:
from CreateLib import Create

MyCreate = Create('/rogers')

MyCreate.beep()
MyCreate.forward(0.5, 0.5)
MyCreate.close()

**More challenges**
- figure out what all the possible colors are
- add speed control to the turn command
- add duration to the beep command
